In [39]:
import requests

inputs = {
    "input": {"messages": ["Puedes buscar un proveedor de servicios spa"]},
    "config": {"configurable": {"thread_id": "", "thread_ts": "string"}},
    "kwargs": {},
}
response = requests.post("http://localhost:8000/bot/invoke", json=inputs)

In [40]:
response.json()

{'output': {'messages': [{'content': 'Puedes buscar un proveedor de servicios spa',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': '3cc073ab-18b2-420e-8877-a264d406968b',
    'example': False},
   {'content': '',
    'additional_kwargs': {'tool_calls': [{'index': 0,
       'id': 'call_1HI8cN7mmPjgYyvXt6evfpfa',
       'function': {'arguments': '{"tag":"spa"}',
        'name': 'get_service_provider_from_firebase'},
       'type': 'function'}]},
    'response_metadata': {'finish_reason': 'tool_calls',
     'model_name': 'gpt-3.5-turbo-0125'},
    'type': 'ai',
    'name': None,
    'id': 'run-e5847648-9332-4bd5-a19c-800d001f5df9-0',
    'example': False,
    'tool_calls': [{'name': 'get_service_provider_from_firebase',
      'args': {'tag': 'spa'},
      'id': 'call_1HI8cN7mmPjgYyvXt6evfpfa'}],
    'invalid_tool_calls': [],
    'usage_metadata': None},
   {'content': '["Carolina", "Angela ", "Emulator 66980917"]',
    'additiona

# Stream

In [41]:
from langserve import RemoteRunnable
from langchain_core.messages import HumanMessage, AIMessage

remote_runnable = RemoteRunnable("http://localhost:8000/bot/")

In [42]:
chat_history = []

while True:
    human = input("Human (Q/q to quit): ")
    if human in {"q", "Q"}:
        print("AI: Bye bye human")
        break

    ai = None
    print("AI: ")
    async for chunk in remote_runnable.astream(
        input={"messages": [human]},
        config={"configurable": {"thread_id": "", "thread_ts": "string"}},
    ):
        # Agent Action
        if "actions" in chunk:
            for action in chunk["actions"]:
                print(
                    f"Calling Tool ```{action['tool']}``` with input ```{action['tool_input']}```"
                )
        # Observation
        elif "steps" in chunk:
            for step in chunk["steps"]:
                print(f"Got result: ```{step['observation']}```")
        # Final result
        elif "output" in chunk:
            print(chunk)
            ai = AIMessage(content=chunk["output"])
        else:
            print(chunk)
        print("------")
    chat_history.extend([HumanMessage(content=human), ai])

AI: 
{'assistant': {'messages': AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_rb69VLCu7nXEMTJAdXZex1YV', 'function': {'arguments': '{}', 'name': 'get_yomap_service_categories'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-b366f183-d98d-4bec-97d3-3d77566c6a4d-0', tool_calls=[{'name': 'get_yomap_service_categories', 'args': {}, 'id': 'call_rb69VLCu7nXEMTJAdXZex1YV'}])}}
------
{'tools': {'messages': [{'content': '["cerrajeria", "piano", "asistente_virtual", "8_56_hierro", "assistant", "tutorias", "muebles", "transporte_de_alimentos", "carrozas", "plomero", "taxi", "tv", "ai", "musica", "depilacion", "llaves_perdidas", "mototaxi", "moda", "asistente", "indriver", "cerrajeria_automotriz", "musica", "nevera", "musico", "piano", "taxi", "ayuda", "aseo", "delivery", "paseo_de_aves", "vegetariano", "health_panama", "iphone", "fisioterapia", "nueva_salud", "costurera", "1", "albanil", "s

In [43]:
def _print_event(event: dict, _printed: set, max_length=1500):
    current_state = event.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])
    message = event.get("messages")
    if message:
        if isinstance(message, list):
            message = message[-1]
        if message.id not in _printed:
            msg_repr = message.pretty_repr(html=True)
            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"
            print(msg_repr)
            _printed.add(message.id)

In [44]:
_printed = set()

In [68]:
chat_history = []

while True:
    human = input("Human (Q/q to quit): ")
    if human in {"q", "Q"}:
        print("AI: Bye bye human")
        break

    ai = None
    print("AI: ")
    async for chunk in remote_runnable.astream(
        input={"messages": [human]},
        config={"configurable": {"thread_id": "", "thread_ts": "string"}},
    ):
        if "assistant" in chunk.keys():
            print(chunk["assistant"]["messages"].content)

AI: 
¡Hola! ¿En qué puedo ayudarte hoy?
AI: 
¡Hola! Para ayudarte a encontrar un servicio de fontanería que pueda solucionar el problema de tu salidero de agua, necesitaría saber en qué ubicación te encuentras. ¿Podrías proporcionarme la ciudad o el código postal donde necesitas este servicio de fontanería?
AI: 
¡Hola! ¿En qué puedo ayudarte hoy?
AI: 

I couldn't find any service providers under the tag "Panam." Let me try searching for service categories related to "Panam" to see if we can find a better match.

It seems there are no service providers listed under the category "health_panama" either. Let's try searching for service providers related to "health" to see if we can find relevant options.
I found some service providers related to "health":

1. Anita
2. Ummed Ali

If you need more information about any of these providers, feel free to ask!
AI: 

No se encontró información sobre Anita en este momento. Permíteme intentar buscar de nuevo.
No se encontraron proveedores de servic

In [67]:
chunk["assistant"]["messages"].content

'Akshdeep is a 15-year-old male service provider located in Sudan, Punjab, India. He specializes in home services. If you need assistance with home-related tasks, Akshdeep may be able to help.'